In [5]:
import numpy as np
import pandas as pd

train = pd.read_csv('train.csv', encoding='utf-8')
test = pd.read_csv('test.csv', encoding='utf-8')
train.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,is_listened
0,25471,1480597215,222606,41774,12,20040704,1,0,223,0,0,9241,55164,29,0
1,25571,1480544735,250467,43941,0,20060301,2,1,171,0,0,16547,55830,30,1
2,16,1479563953,305197,48078,1,20140714,2,1,149,1,1,7665,2704,29,1
3,7,1480152098,900502,71521,0,20001030,0,0,240,0,1,1580,938,30,0
4,7,1478368974,542335,71718,0,20080215,0,0,150,0,1,1812,2939,24,1


In [29]:
train_y = train[['is_listened']].as_matrix()
train_x = train.drop(['is_listened'], axis=1).as_matrix()

In [40]:
import theano
import theano.tensor as T
import lasagne
import os
from lasagne.regularization import l2, l1
from lasagne.regularization import regularize_layer_params


In [41]:
import numpy as np
import scipy

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

def plot_learning_curve(train_curve, test_curve):
    import pylab as plt
    plt.figure(0)
    plt.plot(train_curve)
    plt.plot(test_curve)
    plt.show()

In [42]:
class NetworkGraph(object):
    """
    Neural Net object.
    On initialization creates network and functions to train.
    """
    def __init__(self, args):
        # regularization and dropout rate are tunable parameters
        self.reg = args.reg_rate
        self.dropout = args.drop_rate

        input_var = T.matrix('inputs')
        target_var = T.matrix('targets')

        l_rate = T.scalar()

        self.network, reg = self.build_mlp(input_var,args.template_length)

        # feed forward train output with dropout, evaluate network
        ff_output = lasagne.layers.get_output(self.network)

        # feed forward test run with dropout off (deterministic flag = True)
        ff_test_output = lasagne.layers.get_output(self.network, deterministic=True)

        # define the loss function
        loss = lasagne.objectives.binary_crossentropy(ff_output, target_var)
        loss = loss.mean() + reg

        test_loss = lasagne.objectives.squared_error(ff_test_output, target_var)
        test_loss = test_loss.mean()

        # define which params to update update and update function
        params = lasagne.layers.get_all_params(self.network, trainable=True)
        updates = lasagne.updates.adam(loss, params, learning_rate=l_rate)
        
        self.train_fn = theano.function(inputs=[input_var, target_var, l_rate], outputs=loss, updates=updates, on_unused_input='warn')
        self.val_fn = theano.function([input_var, target_var], outputs=test_loss)
        self.ff_fn = theano.function([input_var], [ff_test_output])
        
    def build_mlp(self, input_var=None, input_length = 14):
        """
        Topology of the network
        :param input_var:
        :return:
        """
        relu = lasagne.nonlinearities.leaky_rectify
        sigmoid = lasagne.nonlinearities.sigmoid

        l_in = lasagne.layers.InputLayer(shape=(None, input_length), input_var=input_var)

        l_in_drop = lasagne.layers.DropoutLayer(l_in, p=self.dropout)

        l_hid1 = lasagne.layers.DenseLayer( \
            l_in_drop, num_units=40, \
            nonlinearity=relu, \
            W=lasagne.init.GlorotUniform() \
            )

        l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=self.dropout)

        l_hid2 = lasagne.layers.DenseLayer( \
            l_hid1_drop, num_units=20, \
            nonlinearity=relu, \
            W=lasagne.init.GlorotUniform() \
            )

        l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=self.dropout)

        l_hid3 = lasagne.layers.DenseLayer( \
            l_hid2_drop, num_units=10, \
            nonlinearity=relu, \
            W=lasagne.init.GlorotUniform() \
            )

        l_hid3_drop = lasagne.layers.DropoutLayer(l_hid3, p=self.dropout)

        self.l_out = lasagne.layers.DenseLayer( \
            l_hid3_drop, num_units=1, \
            nonlinearity=sigmoid, \
            W=lasagne.init.GlorotUniform()
        )

        l1_penalty0 = regularize_layer_params(l_in, l1) * 0.01
        l1_penalty1 = regularize_layer_params(l_hid1, l2) * 0.007

        l1_penalty = l1_penalty0 + l1_penalty1

        return self.l_out, l1_penalty * self.reg

In [43]:
class Model(object):
    """
    Autoencoder model class:
    Train wrapper, Validation wrapper, Load and save network
    """
    def __init__(self, \
                 args, \
                 trX=0, \
                 trY=0, \
                 teX=0, \
                 teY=0, \
                 load_set=1, \
                 ):

        DB_DIR = args.db_dir
        TEMPLATE_LENGTH = 14

        self.trX, self.trY, self.teX = train_x, train_y, test
        self.vaX = self.trX[-1000:]
        self.vaY = self.trY[-1000:]
        
        self.num_epochs = args.num_epochs
        self.net = NetworkGraph(args)
        self.l_rate_val = args.l_rate

    def train_single_epoch(self):
        # pass over the training data:
        train_err = 0
        train_batches = 0
        batch_size = 10
        for batch in iterate_minibatches(self.trX, self.trY, batch_size, shuffle=True):
            inputs, targets = batch
            train_err += self.net.train_fn(inputs, targets, self.l_rate_val * 1.)
            train_batches += 1

        return train_err / train_batches
    def validate_model(self):
        setX, setY = self.vaX, self.vaY


        # pass over the validation data:
        val_err = 0
        val_num_batches = 0
        for batch in iterate_minibatches(setX, setY, 10, shuffle=False):
            inputs, targets = batch
            err = self.net.val_fn(inputs, targets)
            val_err += err

            val_num_batches += 1

        return val_err/ val_num_batches

    def train_model(self):
        import time
        train_curve = []
        test_curve = []

        for epoch in range(self.num_epochs):
            start_time = time.time()

            tr_err = self.train_single_epoch()
            v_err = self.validate_model()

            self.print_train_status(epoch, time.time() - start_time, tr_err, v_err)

            train_curve.append(v_err)
            test_curve.append(tr_err)
            if epoch == 50 or epoch == 100 or epoch == 150:
                self.l_rate_val = self.l_rate_val / 2.

        return train_curve, test_curve
    def print_train_status(self, epoch, timer, tr_err, v_err):
        # print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, self.num_epochs, timer))
        print("  training loss:\t\t{:.6f}".format(tr_err))
        print("  validation loss:\t\t{:.6f}".format(v_err))
        print("  learning rate:\t\t{:.5f} %".format(self.l_rate_val))

    def feed_forward(self, setX):
        """
        Feed forward run of Autoencoder on a given dataset
        :param setX:
        :return:
        """
        ff_output = self.net.ff_fn(setX)
        return ff_output

    def save_model(self):
        try:
            os.stat('models')
        except:
            os.mkdir('models')

        pickle.dump(lasagne.layers.get_all_param_values(self.net.network),
                    open('models/network.pklz', 'wb'))
        print("The network is saved as models/network.pklz")

    def load_model(self, args, model_params=0):
        """
        Load saved model (Neural net)
        :param args:
        :param model_params:
        :return:
        """
        self.net = NetworkGraph(args)
        loadedweights = pickle.load(
            open('models/network.pklz', 'rb'))
        lasagne.layers.set_all_param_values(self.net.network, loadedweights)
        print("The network is loaded.")

In [44]:
import argparse

def arg_parser():
    parser = argparse.ArgumentParser(description="Train classifier")

    parser.add_argument('-db_dir', dest='db_dir', type=str, default='DATA/', help="Mode")
    parser.add_argument('-mode', dest='mode', type=str, default='evaluate', help="Mode")
    parser.add_argument('-l_rate', dest='l_rate', type=float, default=1., help="Learning rate")
    parser.add_argument('-save_load_templ', dest='save_load_templ', type=int, default=0, help="Save template - 0, load - 1")
    parser.add_argument('-num_epochs', dest='num_epochs', type=int, default=10, help="Num epochs")
    parser.add_argument('-num_eval_examples', dest='num_eval', type=int, default=8, help="Number of examples to evaluate")
    parser.add_argument('-reg_rate', dest='reg_rate', type=float, default=0.00001, help="Regularization rate")
    parser.add_argument('-drop_rate', dest='drop_rate', type=float, default=0.05, help="Dropout rate")
    parser.add_argument('-template_length', dest='template_length', type=int, default=14, help="Number of datapoint in template")
    parser.add_argument('-f', dest='f', type=str, default=14, help="")

    return parser.parse_args()



In [ ]:
model = Model(arg_parser())

train_curve, test_curve = model.train_model()

plot_learning_curve(train_curve,test_curve)


[[         0 1478460921  118086546   12266568          2   20151216
           1          0        342          0          1       1485
     1182806         30]
 [         7 1479637287   12351807    1135120          1   20110712
           0          0        173          1          1       4109
       14894         21]
 [       734 1479515785    3616627     341059         13   20070302
           0          0        286          1          1        690
        2519         25]
 [         0 1479586782  132929054   14139248          1   20161021
           0          0        202          1          0       3665
     5542343         26]
 [        17 1478622100   17199784    1610670          1   20101231
           0          0        215          1          1       2267
       71069         30]
 [         7 1479489445    2164908     216679          2   20080929
           1          0        301          0          0       3477
       11907         29]
 [         0 1480100197   97668106